# Exploring data of belkhirAssessingICTGlobal2018

In [1]:
%matplotlib ipympl
import pandas as pd

In {cite}`belkhirAssessingICTGlobal2018` table 9 we find data regarding equipment lifetime and emissions estimates for production and usage per year. The emission numbers are in kg CO<sub>2</sub>eq. The table below excludes the lifecycle annual footprint from the original table. This can be calculated easily: $f_{min} = u_{min} + p_{min}*l_{max}$ and $f_{max} = u_{max} + p_{max}*l_{min}$.

It must be said that *min* and *max* are a bit of a misnomer here. We will use optimistic and pessimistic.

In [2]:
eqem = pd.read_csv("belkhirAssessingICTGlobal2018/table9.csv")
eqem

,equipment,useful life min,useful life max,prod E min,prod E max,yearly use E min,yearly use E max
0,Home desktop,5,7,218,628,93.0,116.00
1,Office desktop,5,7,218,628,69.0,75.00
2,Home notebook,5,7,281,468,27.0,35.00
3,Office notebook,5,7,281,468,20.0,23.00
4,CRT display,5,7,200,200,51.0,95.00
5,LCD display,5,7,95,95,23.0,43.00
6,Tablet,3,8,80,116,4.5,5.25
7,Smart phones,2,2,40,80,4.5,5.25
8,Datacenter server,3,5,328,328,1314.0,3743.00


Using the formulas above we compute the yearly production emissions and total footprint:

In [3]:
eqem["yearly prod E opt"] = eqem["prod E min"]/eqem["useful life max"]
eqem["yearly total E opt"] = eqem["yearly prod E opt"]+eqem["yearly use E min"]

eqem[["equipment","yearly total E opt"]]

,equipment,yearly total E opt
0,Home desktop,124.142857
1,Office desktop,100.142857
2,Home notebook,67.142857
3,Office notebook,60.142857
4,CRT display,79.571429
5,LCD display,36.571429
6,Tablet,14.500000
7,Smart phones,24.500000
8,Datacenter server,1379.600000


Take equipment quantity data of 2020 from {cite}`belkhirAssessingICTGlobal2018` table 10:

In [12]:
qtn = pd.read_csv("belkhirAssessingICTGlobal2018/table10.csv", index_col="Year")
qtn2020 = qtn.loc[[2020]]
qtn

,Home desktop,Office desktop,Office notebook,Home notebook,CRT display,LCD display,Tablet,Smart phones
Year,,,,,,,,
2007,276,414,130,195,459,594,NaN,200
2008,291,437,167,250,250,512,NaN,271
2009,301,451,207,311,233,631,NaN,342
2010,312,468,258,386,216,750,NaN,485
2011,315,472,310,464,199,870,NaN,771
2012,305,458,359,538,168,1482,NaN,1136
2013,308,462,376,564,166,1108,660.0,1580
2014,307,460,391,586,149,1227,860.0,1833
2015,300,449,397,596,132,1346,1000.0,2246


Compute the optimistic total emissions for 2020 using the quantity and equipment energy data

In [5]:
eqem2020 = eqem[["equipment", "yearly total E opt", "yearly use E min", "yearly prod E opt"]].join(qtn2020.T.rename(columns={2020: "quantity"}),on="equipment")
eqem2020

,equipment,yearly total E opt,yearly use E min,yearly prod E opt,quantity
0,Home desktop,124.142857,93.0,31.142857,255.0
1,Office desktop,100.142857,69.0,31.142857,383.0
2,Home notebook,67.142857,27.0,40.142857,625.0
3,Office notebook,60.142857,20.0,40.142857,417.0
4,CRT display,79.571429,51.0,28.571429,48.0
5,LCD display,36.571429,23.0,13.571429,1942.0
6,Tablet,14.500000,4.5,10.000000,1460.0
7,Smart phones,24.500000,4.5,20.000000,3619.0
8,Datacenter server,1379.600000,1314.0,65.600000,NaN


In [6]:
eqem2020["total opt"] = eqem2020["quantity"]*eqem2020["yearly total E opt"]/1000
eqem2020["total opt %"] = eqem2020["total opt"]/eqem2020["total opt"].sum()

eqem2020["use opt"] = eqem2020["quantity"]*eqem2020["yearly use E min"]/1000
eqem2020["use opt %"] = eqem2020["use opt"]/eqem2020["use opt"].sum()

eqem2020["prod opt"] = eqem2020["quantity"]*eqem2020["yearly prod E opt"]/1000
eqem2020["prod opt %"] = eqem2020["prod opt"]/eqem2020["prod opt"].sum()

eqem2020

,equipment,yearly total E opt,yearly use E min,yearly prod E opt,quantity,total opt,total opt %,use opt,use opt %,prod opt,prod opt %
0,Home desktop,124.142857,93.0,31.142857,255.0,31.656429,0.098394,23.7150,0.163184,7.941429,0.045018
1,Office desktop,100.142857,69.0,31.142857,383.0,38.354714,0.119213,26.4270,0.181846,11.927714,0.067615
2,Home notebook,67.142857,27.0,40.142857,625.0,41.964286,0.130433,16.8750,0.116118,25.089286,0.142225
3,Office notebook,60.142857,20.0,40.142857,417.0,25.079571,0.077952,8.3400,0.057388,16.739571,0.094893
4,CRT display,79.571429,51.0,28.571429,48.0,3.819429,0.011871,2.4480,0.016845,1.371429,0.007774
5,LCD display,36.571429,23.0,13.571429,1942.0,71.021714,0.220748,44.6660,0.307349,26.355714,0.149404
6,Tablet,14.500000,4.5,10.000000,1460.0,21.170000,0.065800,6.5700,0.045209,14.600000,0.082764
7,Smart phones,24.500000,4.5,20.000000,3619.0,88.665500,0.275588,16.2855,0.112061,72.380000,0.410305
8,Datacenter server,1379.600000,1314.0,65.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To get a complete picture of the emissions in ICT Belkhir and Elmeligi provide historical and projected emissions data for data center and networks in Mt-CO<sub>2</sub>eq in table 11 {cite}`belkhirAssessingICTGlobal2018`. The numbers lack production emissions, but the usage provides a better estimate than using the data for datacenter server. Belkhir and Elmeligi argue that the emissions of production of datacenter equipment is negligible compared to its usage, which might be true, but given production emissions for the other equipment is not that high, so in total the emissions contribution of datacenter equipment production might be higher than of some office equipment.

In [7]:
clem = pd.read_csv("belkhirAssessingICTGlobal2018/table11.csv", index_col="Year")
clem

,data center,networks
Year,,
2007,113.4,101.5
2008,127.0,114.0
2009,142.3,127.0
2010,159.3,138.0
2011,178.5,152.0
2012,199.9,167.0
2013,223.9,178.6
2014,250.7,191.5
2015,280.8,204.4


We will construct a table with optimistic emission data on 2020 per equipment and life cycle stage (use or prod). This will give an indication on emission per equipment type and per life cycle stage.

In [8]:
el2020 = (eqem2020[["equipment","use opt","prod opt"]]
          .dropna()
          .set_index("equipment")
          .rename(columns={
              "use opt": "use", 
              "prod opt": "prod"
            })
        )
el2020 = el2020.stack().rename(2020).rename_axis(index={None:"life cycle"})
el2020 = el2020.to_frame().reset_index()
el2020

,equipment,life cycle,2020
0,Home desktop,use,23.715000
1,Home desktop,prod,7.941429
2,Office desktop,use,26.427000
3,Office desktop,prod,11.927714
4,Home notebook,use,16.875000
5,Home notebook,prod,25.089286
6,Office notebook,use,8.340000
7,Office notebook,prod,16.739571
8,CRT display,use,2.448000
9,CRT display,prod,1.371429


This table lacks datacenter and network data, so let's add those, giving us a table with optimistic emission data for 2020 in Mt CO<sub>2</sub>eq:

In [9]:
cl2020 = clem.T[[2020]]
cl2020["life cycle"]="use"
cl2020.index.name = "equipment"
cl2020 = cl2020.reset_index()
emissions2020 = el2020.merge(cl2020,how="outer")
emissions2020

,equipment,life cycle,2020
0,Home desktop,use,23.715000
1,Home desktop,prod,7.941429
2,Office desktop,use,26.427000
3,Office desktop,prod,11.927714
4,Home notebook,use,16.875000
5,Home notebook,prod,25.089286
6,Office notebook,use,8.340000
7,Office notebook,prod,16.739571
8,CRT display,use,2.448000
9,CRT display,prod,1.371429


Even lacking numbers for manufacturing emissions of data centers and networks, they clearly dominate the emissions landscape. The second figure better depicts the ration between emissions caused by manufacturing and usage. These numbers are a stark contradiction to those presented in {cite}`inrWeNR2021Release2021`.

In [10]:
import plotly.express as px

fig = px.sunburst(emissions2020, path=["equipment", "life cycle"], values=2020)
fig.update_traces(textinfo="label+percent root")
fig.show()

In [11]:
fig2 = px.sunburst(emissions2020, path=["life cycle", "equipment"], values=2020)
fig2.update_traces(textinfo="label+percent root")
fig2.show()